## Градиентный спуск с постоянным шагом

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [15]:
from Functions.user_functions import f_well, f_poor, f_rozen, grad_well, grad_poor, grad_rozen
from Functions.optimization import classic_grad_descent

In [16]:
N_LR = 5
lr_classic_grad_descent = np.empty((N_LR,), dtype=object)
for n in range(N_LR):
    lr_classic_grad_descent[n] = np.array([1/10**(n+1+2), 1/10**(n+1+2)])
    
for lr in lr_classic_grad_descent:
    print(lr)

[0.001 0.001]
[0.0001 0.0001]
[1.e-05 1.e-05]
[1.e-06 1.e-06]
[1.e-07 1.e-07]


In [17]:
lr_classic_grad_descent = np.delete(lr_classic_grad_descent, [0,1,4])
for lr in lr_classic_grad_descent:
    print(lr)
N_LR = len(lr_classic_grad_descent)

[1.e-05 1.e-05]
[1.e-06 1.e-06]


In [18]:
N_EPS = 5
EPS_INIT = -1
eps_var = np.logspace(EPS_INIT, EPS_INIT-N_EPS+1, N_EPS)
print(eps_var)

[1.e-01 1.e-02 1.e-03 1.e-04 1.e-05]


In [19]:
func_names = [f_well, f_poor, f_rozen]
grad_names = [grad_well, grad_poor, grad_rozen]
func_labels = ['Well-conditioned',
               'Poorly-conditioned',
               'Rosenbrock']

N_FUNC = len(func_names)

In [20]:
x0_optim_arr     = np.empty((N_FUNC, N_LR), dtype='object')
x1_optim_arr     = np.empty((N_FUNC, N_LR), dtype='object')
iter_counter_arr = np.empty((N_FUNC, N_LR), dtype='object')
func_counter_arr = np.empty((N_FUNC, N_LR), dtype='object')
grad_counter_arr = np.empty((N_FUNC, N_LR), dtype='object')

# Начальное приближение.
x_init = np.array([2.2, -2.2])

for ndx, func, grad in zip(range(N_FUNC), func_names, grad_names):
    for mdx, lr in enumerate(lr_classic_grad_descent):
        x0_optim_list     = []
        x1_optim_list     = []
        iter_counter_list = []
        func_counter_list = []
        grad_counter_list = []
    
        for idx, eps in enumerate(eps_var):
            x_optim, _, iter_final, func_counter, grad_counter = classic_grad_descent(
                                                                                        loss_func=func,
                                                                                        grad_func=grad,
                                                                                        x_init=x_init,
                                                                                        learning_rate=lr,
                                                                                        tolerance=eps,
                                                                                        printoutput=False
                                                                                        )
            x0_optim_list.append(x_optim[0])
            x1_optim_list.append(x_optim[1])
            iter_counter_list.append(iter_final)
            func_counter_list.append(func_counter)
            grad_counter_list.append(grad_counter)
                       
        x0_optim_arr[ndx, mdx] = x0_optim_list
        x1_optim_arr[ndx, mdx] = x1_optim_list
        iter_counter_arr[ndx, mdx] = iter_counter_list
        func_counter_arr[ndx, mdx] = func_counter_list
        grad_counter_arr[ndx, mdx] = grad_counter_list

Преобразуем в таблицы.

In [21]:
col_name = []
# Special column names for x_optim.
col_name_x_optim = []
for idx in range(N_LR):
    col_name.append('x1: lr=' + str(lr_classic_grad_descent[idx][0]) + ', x2: lr=' + str(lr_classic_grad_descent[idx][1]))
    col_name_x_optim.append('x1 (lr=' + str(lr_classic_grad_descent[idx][0]) + ')')
    col_name_x_optim.append('x2 (lr=' + str(lr_classic_grad_descent[idx][1]) + ')')

x_optim_tbl_set      = np.empty((N_FUNC,), dtype=object)
iter_counter_tbl_set = np.empty((N_FUNC,), dtype=object)
func_counter_tbl_set = np.empty((N_FUNC,), dtype=object)
grad_counter_tbl_set = np.empty((N_FUNC,), dtype=object)

for ndx in range(N_FUNC):
    x_optim_tbl      = eps_var.copy()
    iter_counter_tbl = eps_var.copy()
    func_counter_tbl = eps_var.copy()
    grad_counter_tbl = eps_var.copy()
    
    for idx, colx1, colx2, col_iter, col_func, col_grad in zip(
                                                                range(N_LR),
                                                                x0_optim_arr[ndx, :],
                                                                x1_optim_arr[ndx, :],
                                                                iter_counter_arr[ndx, :],
                                                                func_counter_arr[ndx, :],
                                                                grad_counter_arr[ndx, :]
                                                                ):
        x_optim_tbl      = np.vstack((x_optim_tbl, colx1, colx2))
        iter_counter_tbl = np.vstack((iter_counter_tbl, col_iter))
        func_counter_tbl = np.vstack((func_counter_tbl, col_func))
        grad_counter_tbl = np.vstack((grad_counter_tbl, col_grad))
    
    x_optim_tbl_set[ndx] = pd.DataFrame(x_optim_tbl.T, columns=['eps'] + col_name_x_optim)

    iter_counter_tbl_set[ndx] = pd.DataFrame(iter_counter_tbl.T, columns=['eps'] + col_name)
    iter_counter_tbl_set[ndx][col_name] = iter_counter_tbl_set[ndx][col_name].astype(int)

    func_counter_tbl_set[ndx] = pd.DataFrame(func_counter_tbl.T, columns=['eps'] + col_name)
    func_counter_tbl_set[ndx][col_name] = func_counter_tbl_set[ndx][col_name].astype(int)
    
    grad_counter_tbl_set[ndx] = pd.DataFrame(grad_counter_tbl.T, columns=['eps'] + col_name)
    grad_counter_tbl_set[ndx][col_name] = grad_counter_tbl_set[ndx][col_name].astype(int)

In [22]:
for ndx, lbl in enumerate(func_labels):
    print(lbl)
    display(x_optim_tbl_set[ndx])

Well-conditioned


,eps,x1 (lr=1e-05),x2 (lr=1e-05),x1 (lr=1e-06),x2 (lr=1e-06)
0,0.10000,0.063774,-0.028077,0.063775,-0.028077
1,0.01000,0.006431,-0.002691,0.006432,-0.002691
2,0.00100,0.000644,-0.000267,0.000644,-0.000267
3,0.00010,0.000064,-0.000027,0.000064,-0.000027
4,0.00001,0.000006,-0.000003,0.000006,-0.000003


Poorly-conditioned


,eps,x1 (lr=1e-05),x2 (lr=1e-05),x1 (lr=1e-06),x2 (lr=1e-06)
0,0.10000,-5.492850,-0.207137,-5.492849,-0.207137
1,0.01000,-5.516818,-0.207727,-5.516817,-0.207727
2,0.00100,-5.519214,-0.207786,-5.519214,-0.207786
3,0.00010,-5.519454,-0.207792,-5.519454,-0.207792
4,0.00001,-5.519478,-0.207792,-5.519478,-0.207792


Rosenbrock


,eps,x1 (lr=1e-05),x2 (lr=1e-05),x1 (lr=1e-06),x2 (lr=1e-06)
0,0.10000,0.897282,0.804678,0.897281,0.804677
1,0.01000,0.988919,0.977916,0.988919,0.977916
2,0.00100,0.998883,0.997763,0.994403,0.988815
3,0.00010,0.999888,0.999776,0.994403,0.988815
4,0.00001,0.999989,0.999978,0.994403,0.988815


In [23]:
for ndx, lbl in enumerate(func_labels):
    print(lbl)
    display(iter_counter_tbl_set[ndx])

Well-conditioned


,eps,"x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06"
0,0.10000,259350,2593507
1,0.01000,419832,4198335
2,0.00100,580365,5803676
3,0.00010,740900,7409030
4,0.00001,901434,9014384


Poorly-conditioned


,eps,"x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06"
0,0.10000,150842,1508436
1,0.01000,212181,2121828
2,0.00100,273519,2735221
3,0.00010,334857,3348614
4,0.00001,396195,3962007


Rosenbrock


,eps,"x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06"
0,0.10000,309189,3091716
1,0.01000,831155,8311392
2,0.00100,1401746,10000000
3,0.00010,1977709,10000000
4,0.00001,2554215,10000000


In [24]:
for ndx, lbl in enumerate(func_labels):
    print(lbl)
    display(func_counter_tbl_set[ndx])

Well-conditioned


,eps,"x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06"
0,0.10000,259350,2593507
1,0.01000,419832,4198335
2,0.00100,580365,5803676
3,0.00010,740900,7409030
4,0.00001,901434,9014384


Poorly-conditioned


,eps,"x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06"
0,0.10000,150842,1508436
1,0.01000,212181,2121828
2,0.00100,273519,2735221
3,0.00010,334857,3348614
4,0.00001,396195,3962007


Rosenbrock


,eps,"x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06"
0,0.10000,309189,3091716
1,0.01000,831155,8311392
2,0.00100,1401746,10000000
3,0.00010,1977709,10000000
4,0.00001,2554215,10000000


In [25]:
for ndx, lbl in enumerate(func_labels):
    print(lbl)
    display(grad_counter_tbl_set[ndx])

Well-conditioned


,eps,"x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06"
0,0.10000,259350,2593507
1,0.01000,419832,4198335
2,0.00100,580365,5803676
3,0.00010,740900,7409030
4,0.00001,901434,9014384


Poorly-conditioned


,eps,"x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06"
0,0.10000,150842,1508436
1,0.01000,212181,2121828
2,0.00100,273519,2735221
3,0.00010,334857,3348614
4,0.00001,396195,3962007


Rosenbrock


,eps,"x1: lr=1e-05, x2: lr=1e-05","x1: lr=1e-06, x2: lr=1e-06"
0,0.10000,309189,3091716
1,0.01000,831155,8311392
2,0.00100,1401746,10000000
3,0.00010,1977709,10000000
4,0.00001,2554215,10000000


In [13]:
plot_list = [x_optim_df, iter_counter_df, func_counter_df]
plot_name = ['x-координата минимума функции', 'Количество итераций', 'Количество вычислений функции']
fig_name = ['x_optim', 'iter_number', 'func_calc_number']

tbl_title = [0] * 3

tbl_title[0] = 'Таблица оптимальных значений (' + func_comment + ' функция)'
tbl_title[1] = 'Таблица количества итераций (' + func_comment + ' функция)'
tbl_title[2] = 'Таблица количества вычислений функции (' + func_comment + ' функция)'

for tbl_tlt, plt_lst in zip(tbl_title, plot_list):
    print(tbl_tlt)
    display(plt_lst)

NameError: name 'x_optim_df' is not defined

In [ ]:
#grad_descent_methods = [classic_grad_descent]#, armijo_grad_descent, steep_grad_descent]
#N_METH = len(grad_descent_methods)

